In [1]:
!pip install quandl


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [5]:
pip install cryptocompare


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
import requests 
import pandas as pd
import quandl
from datetime import datetime

# Define financial data parameters
xt_dataset_code = 'FRED/FEDFUNDS'
yt_fsym = 'BTC'
yt_tsym = 'USD'
yt_api_key = 'hC2PHkLKcHsPzxPRcBwK'  # replace with your API key

In [15]:
start_date = '2016-01-01'
end_date = '2021-01-01'
xt_data = quandl.get(xt_dataset_code, start_date=start_date, end_date=end_date, column_index=1)
print(xt_data)

            Value
Date             
2016-01-01   0.34
2016-02-01   0.38
2016-03-01   0.36
2016-04-01   0.37
2016-05-01   0.37
...           ...
2020-09-01   0.09
2020-10-01   0.09
2020-11-01   0.09
2020-12-01   0.09
2021-01-01   0.09

[61 rows x 1 columns]


In [16]:
# Fetch financial data from CryptoCompare
url = "https://min-api.cryptocompare.com/data/v2/histoday"
data_format = '%Y-%m-%d'
days = (datetime.strptime(end_date, data_format) - datetime.strptime(start_date, data_format)).days
params = {
    'fsym': yt_fsym,
    'tsym': yt_tsym,
    'limit': days,
    'toTs': int(datetime.strptime(end_date, data_format).timestamp()),
    'api_key': yt_api_key
}

In [17]:
response = requests.get(url, params=params)
response_json = response.json()

In [18]:
yt_data_raw = response_json['Data']['Data']
yt_data = pd.DataFrame.from_records(yt_data_raw)
yt_data['time'] = pd.to_datetime(yt_data['time'], unit='s')
yt_data.set_index('time', inplace=True)
yt_data = yt_data[['close']]
data = xt_data.join(yt_data, how='inner')

In [19]:
yt_data = data['close']
print(yt_data)

2016-01-01      433.99
2016-02-01      372.18
2016-03-01      434.04
2016-04-01      415.64
2016-05-01      452.24
                ...   
2020-09-01    11926.69
2020-10-01    10622.79
2020-11-01    13761.72
2020-12-01    18792.03
2021-01-01    29388.94
Name: close, Length: 61, dtype: float64


In [20]:
data= xt_data.join(yt_data, how= 'inner')
xt= data.iloc[:,0].values
yt= data.iloc[:,1].values

In [21]:
import numpy as np
xt=np.array(xt)
yt=np.array(yt)

x_mean=np.mean(xt)
y_mean= np.mean(yt)

In [22]:
numerator = np.sum((xt-x_mean)*(yt-y_mean))
denominator=np.sum((xt-x_mean)**2)

In [23]:
beta=numerator/denominator
alpha=y_mean-beta*x_mean

In [24]:
print(f"Alpha:{alpha}")
print(f"Beta:{beta}")

Alpha:6219.797367050046
Beta:58.83724162257572


In [25]:
def gradient_descent(x, y, alpha=0.001, beta=0.001, learning_rate=0.001, num_iterations=1000):
    n = len(x)
    for _ in range(num_iterations):
        y_predicted= alpha+beta*x
        d_alpha=-(2/n)*np.sum(y-y_predicted)
        d_beta=-(2/n)*np.sum(x*(y-y_predicted))
        alpha=learning_rate*d_alpha
        beta=learning_rate*d_beta
    return alpha,beta
alpha_gd,beta_gd=gradient_descent(xt,yt)

In [26]:
print(f"Alpha:{alpha_gd}")
print(f"Beta:{beta_gd}")

Alpha:-12.631191886831406
Beta:-14.43269559880266


Stochastic Gradient Descent (SGD) is another variant of Gradient Descent. When the dataset is sufficiently large and computing the gradient across the full dataset would be computationally expensive, SGD is utilised. Based on the gradient of the loss function for each sample in the dataset, SGD modifies the model's parameters. In comparison to batch gradient descent, this leads to faster convergence and reduced memory needs, but it can be noisier and take more iterations to converge to a minimum.

Convergence to a loss function minimum could be challenging to achieve in practise for a number of different reasons.  The occurrence of local minima, where the optimisation algorithm becomes stuck in a poor solution, is a typical problem.  The existence of saddle points, when the gradient is almost zero in all directions, is another problem, making it challenging to identify the direction of the steepest fall.  The optimisation landscape can also not be convex and contain a lot of flat sections, which would hinder convergence.

To overcome these issues, several techniques have been developed, such as using momentum to accelerate convergence, adaptive learning rate methods to adjust the step size based on the curvature of the loss function, and regularization techniques such as L1/L2 regularization and dropout to prevent overfitting. Additionally, using more advanced optimization techniques such as Adam and Adagrad can also improve convergence to a minimum. In practice, it is often necessary to experiment with different optimization techniques and hyperparameters to find the optimal solution.




question 3

1.

Bitcoin is not Turing complete according to the mainstream view. This is because it has limited scripting language capabilities, which can only support a set of predetermined functions. These functions are designed to ensure the security and consistency of the Bitcoin network by enforcing rules such as limiting the number of bitcoins that can be created, verifying transactions, and preventing double-spending.

Ian Grigg, in his paper "Financial Cryptography in 7 Layers," argues that Bitcoin is Turing complete. Grigg believes that the script language used by Bitcoin, which is based on the Forth programming language, is sufficiently flexible to allow for the implementation of any computation. He notes that while some operations are limited, they can be overcome through creative programming.

However, the majority of mainstream computer science specialists do not share Grigg's perspective. Critics claim that Turing completeness isn't achieved by Bitcoin's scripting language because it lacks key components like conditional branching and looping, which enable complicated calculations. The limited stack size and defined execution constraints also place additional restrictions on the language's possibilities.

I believe that Bitcoin is not Turing complete.Even while there may be certain computations that can be done within the limitations of the scripting language, these are few and insufficient to satisfy the requirements of Turing completeness. 

In general, experts are still debating the Turing completeness of Bitcoin and researching the strengths and weaknesses of the scripting language.





2.

Traditional chatbots and ChatGPT are different in a number of ways.  First off, ChatGPT is based on the GPT-3.5 architecture, a cutting-edge language model that can provide responses that are human-like for a variety of inputs.  The ability of classic chatbots to handle complicated or unexpected inputs is constrained because they rely on rules-based or decision-tree techniques.  Second, ChatGPT has a wide knowledge base that it can use to construct responses because it was trained on a sizable corpus of text data.  Traditional chatbots, on the other hand, are frequently trained on a smaller range of data and may have trouble with new or obscure topics. Finally, ChatGPT can learn and adapt over time based on user interactions, whereas traditional chatbots are typically static and do not improve with use.

The potential benefits of using a large language model like ChatGPT for conversational AI are significant. To begin with, compared to standard chatbots, ChatGPT can produce responses that are more interesting and natural-sounding, which can increase user happiness and engagement.  Furthermore, ChatGPT is more adaptable than conventional chatbots since it can handle a variety of themes and inputs. Addtionally , ChatGPT has the capacity to grow and change with use, enhancing both its efficiency and accuracy.


However, there are also potential drawbacks to using a large language model like ChatGPT for conversational AI.  Initially, ChatGPT may produce offensive or biased responses as a result of its training data, which might undermine user confidence and reputation.  Moreover, ChatGPT can be resource-intensive, necessitating a lot of processing power and storage to function properly.  Finally, ChatGPT's accuracy and dependability may be jeopardised by malicious assaults such as data poisoning or adversarial inputs.










3.

Blockchain technology has been proposed as a potential solution to some of the systemic problems that can arise from AI, particularly in the areas of transparency, accountability, and data privacy.   Ian Grigg, in his paper "Financial Cryptography in 7 Layers," discusses the use of blockchain technology to create "smart contracts" that can facilitate automated transactions and enforce rules and regulations in a transparent and secure manner.

Increased transparency is one of the key advantages of combining AI and blockchain technology.It is now possible to track and verify the origin and use of data by documenting all transactions and data inputs on a distributed ledger, guaranteeing that AI systems are functioning within moral and legal bounds. By enabling the development of audit trails and accountability procedures that may be used to recognise and address problems with bias, inaccuracy, or wrongdoing, blockchain technology can also promote more accountability.

However, there are also some limitations to the use of blockchain technology in the context of AI. For example, blockchain technology can be computationally expensive, which can limit its scalability and usability for large-scale AI systems.   Additionally, blockchain technology may not be well-suited for all types of AI applications, particularly those that require real-time processing or decision-making.

In conclusion, while blockchain technology has the potential to address some of the systemic issues related to AI, its success will depend on a variety of variables, including the particular use case, the design of the blockchain system, and the accessibility of computational resources.   To fully comprehend the potential advantages and disadvantages of utilising blockchain technology for AI, as well as to establish best practices for its application, more investigation and development will be required.







question 4

I have chosen to analyze the daily closing price of the S&P 500 Index from January 1, 2010, to April 1, 2023. I have chosen this financial time series because the S&P 500 Index is widely used as a benchmark for the overall performance of the US stock market. The S&P 500 Index includes 500 large-cap stocks traded in the US and has a broad representation of the economy, including sectors such as technology, healthcare, finance, and energy.

The S&P 500 Index is impacted by a number of variables, including macroeconomic ones like GDP growth, interest rates, and macroeconomic variables like inflation. Corporate earnings, investor sentiment, geopolitical developments, and global market movements are other crucial variables. The performance of certain index firms also has a big impact on the index's overall performance. In general, the S&P 500 Index tends to increase when corporations post great results or when there is good news about the economy, and vice versa.

In [27]:
import pandas as pd
import numpy as np
import quandl

# Load S&P 500 Index data
quandl.ApiConfig.api_key ='hC2PHkLKcHsPzxPRcBwK'
data = quandl.get("MULTPL/SP500_REAL_PRICE_MONTH")

# Resample to daily frequency
data = data.resample('D').last().fillna(method='ffill')

In [28]:
data['Returns'] = data['Value'].pct_change()

In [29]:
data['Returns_Lag1'] = data['Returns'].shift(1)
data['Returns_Lag2'] = data['Returns'].shift(2)
data['Returns_Lag3'] = data['Returns'].shift(3)

In [30]:
data['Returns_Rolling5'] = data['Returns'].rolling(5).mean().shift(1)
data['Returns_Rolling10'] = data['Returns'].rolling(10).mean().shift(1)
data['Returns_Rolling20'] = data['Returns'].rolling(20).mean().shift(1)

In [31]:
data['Value_Rolling20'] = data['Value'].rolling(20).mean().shift(1)
data['Value_Rolling50'] = data['Value'].rolling(50).mean().shift(1)

In [32]:
data = data.dropna()
X = data.drop(['Value', 'Returns'], axis=1)
y = data['Returns']
split = int

In [33]:
print(X)
print(y)

            Returns_Lag1  Returns_Lag2  Returns_Lag3  Returns_Rolling5  \
Date                                                                     
1871-02-20      0.000000           0.0           0.0          0.000000   
1871-02-21      0.000000           0.0           0.0          0.000000   
1871-02-22      0.000000           0.0           0.0          0.000000   
1871-02-23      0.000000           0.0           0.0          0.000000   
1871-02-24      0.000000           0.0           0.0          0.000000   
...                  ...           ...           ...               ...   
2023-03-28      0.000000           0.0           0.0          0.000000   
2023-03-29      0.000000           0.0           0.0          0.000000   
2023-03-30      0.000000           0.0           0.0          0.000000   
2023-03-31      0.000000           0.0           0.0          0.000000   
2023-04-01      0.035466           0.0           0.0          0.007093   

            Returns_Rolling10  Return